In [1]:
from stable_baselines3 import DDPG

from rl_training.two_wheel_robot import TwoWheelRobot

env = TwoWheelRobot()

model = DDPG("MlpPolicy", env, verbose=1)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/home/amir/.local/share/virtualenvs/python-rQ1OrFAJ/lib/python3.9/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  ../c10/cuda/CUDAFunctions.cpp:115.)
  return torch._C._cuda_getDeviceCount() > 0


In [2]:
obs = env.reset()
env.render()
model.learn(total_timesteps=1000)

for i in range(1000):
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, done, info = env.step(action)
    env.render()
    if done:
        break

Creating window glfw
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 34       |
|    ep_rew_mean     | -324     |
| time/              |          |
|    episodes        | 4        |
|    fps             | 172      |
|    time_elapsed    | 0        |
|    total timesteps | 136      |
| train/             |          |
|    actor_loss      | 8.54     |
|    critic_loss     | 341      |
|    learning_rate   | 0.001    |
|    n_updates       | 46       |
---------------------------------
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 53.4      |
|    ep_rew_mean     | -1.98e+03 |
| time/              |           |
|    episodes        | 8         |
|    fps             | 93        |
|    time_elapsed    | 4         |
|    total timesteps | 427       |
| train/             |           |
|    actor_loss      | 72.1      |
|    critic_loss     | 130       |
|    learning_rate   | 0.001     |
|    n_updates

In [11]:
import torch


<generator object Module.parameters at 0x7f754cc61eb0>

In [12]:
list(model.actor.parameters())[0]

Parameter containing:
tensor([[-0.0325, -0.2052, -0.1637,  ..., -0.0488, -0.0085, -0.0538],
        [-0.2470, -0.2832, -0.1533,  ..., -0.0477,  0.0011, -0.1141],
        [-0.0016, -0.0412, -0.1075,  ..., -0.1612, -0.1489, -0.1568],
        ...,
        [-0.0604, -0.1436, -0.0283,  ...,  0.0635,  0.0180, -0.1484],
        [-0.0730, -0.1594,  0.0052,  ..., -0.0664, -0.0435,  0.0154],
        [-0.1458, -0.1736, -0.0079,  ..., -0.1783, -0.2089,  0.0598]],
       requires_grad=True)

In [15]:
x = torch.randn(1, len(obs), requires_grad=False, device="cuda")

In [16]:
torch.onnx.export_to_pretty_string(model.actor, x, "actor_model.onnx")


'ModelProto {\n  producer_name: "pytorch"\n  domain: ""\n  doc_string: ""\n  graph:\n    GraphProto {\n      name: "torch-jit-export"\n      inputs: [{name: "obs", type:Tensor dtype: 1, Tensor dims: 1 45}]\n      outputs: [{name: "14", type:Tensor dtype: 1, Tensor dims: 1 2}]\n      value_infos: []\n      initializers: [TensorProto shape: [400 45],TensorProto shape: [400],TensorProto shape: [300 400],TensorProto shape: [300],TensorProto shape: [2 300],TensorProto shape: [2]]\n      nodes: [\n        Node {type: "Cast", inputs: [obs], outputs: [7], attributes: [{ name: \'to\', type: int, value: 1}]},\n        Node {type: "Flatten", inputs: [7], outputs: [8], attributes: [{ name: \'axis\', type: int, value: 1}]},\n        Node {type: "Gemm", inputs: [8,mu.0.weight,mu.0.bias], outputs: [9], attributes: [{ name: \'alpha\', type: float, value: 1},{ name: \'beta\', type: float, value: 1},{ name: \'transB\', type: int, value: 1}]},\n        Node {type: "Relu", inputs: [9], outputs: [10], attr

In [17]:
torch.onnx.export(model.actor, x, "actor_model.onnx")


In [3]:
onnx_model = onnx.load("actor_model.onnx")  # load onnx model

dims = onnx_model.graph.input[0].type.tensor_type.shape.dim


from functools import reduce
raw_dims = [x.dim_value for x in dims]

size = reduce((lambda x, y: x * y), raw_dims)

In [4]:
size

NameError: name 'size' is not defined

In [5]:
import struct

struct.unpack_from('2f',onnx_model.graph.initializer[5].raw_data)



(0.03371463716030121, -0.020571403205394745)

In [7]:
import numpy as np
with open("weight_definitions.h", "w") as header_file:
    for i, initializer in enumerate(onnx_model.graph.initializer):
        format_spec = f"{len(initializer.raw_data)//4}f"
        with np.printoptions(floatmode='unique') as po:
           float_data = np.array(struct.unpack_from(format_spec,initializer.raw_data))
            
        raw_def = f""" float {initializer.name.replace('.','_')}[] {{ 
        {np.array2string(float_data, separator=',',floatmode='unique', threshold=10000000,  )};
}};
        """
        header_file.write(raw_def)
        print(raw_def)
    

 float mu_0_weight[] { 
        [ 6.7014120519161224e-02,-1.6942365095019341e-02, 3.7594266235828400e-02,
  2.4409018456935883e-02, 4.7182772308588028e-02, 5.0773489056155086e-04,
 -1.0694573074579239e-01, 6.5682351589202881e-02, 1.5995161235332489e-01,
  2.0223636925220490e-01,-6.0133736580610275e-02, 2.2090613842010498e-02,
  2.9235121607780457e-01, 6.4794935286045074e-02, 2.7194652706384659e-02,
 -9.0976955834776163e-04, 1.3673675060272217e-01,-1.1275560408830643e-01,
  2.1994138136506081e-02, 2.7475396171212196e-02,-3.5156678408384323e-02,
  9.2564322054386139e-02, 2.4926743935793638e-03, 1.5943581238389015e-02,
  1.2685646116733551e-01,-1.6050063073635101e-02,-4.8550114035606384e-02,
  1.2327282130718231e-01,-1.0483999550342560e-01, 1.2847893871366978e-02,
 -5.3394292481243610e-03,-9.8649464547634125e-02, 8.6282491683959961e-02,
  1.7841340601444244e-01,-1.5647771954536438e-01, 7.7998354099690914e-03,
  6.8063452839851379e-02, 2.2944453358650208e-01, 1.6501615941524506e-01,
 -5.21

In [16]:
struct.unpack_from("f",initializer.raw_data)

(-0.020571403205394745,)

In [10]:
onnx_model.graph.node

[input: "obs"
output: "7"
name: "Cast_0"
op_type: "Cast"
attribute {
  name: "to"
  i: 1
  type: INT
}
, input: "7"
output: "8"
name: "Flatten_1"
op_type: "Flatten"
attribute {
  name: "axis"
  i: 1
  type: INT
}
, input: "8"
input: "mu.0.weight"
input: "mu.0.bias"
output: "9"
name: "Gemm_2"
op_type: "Gemm"
attribute {
  name: "alpha"
  f: 1.0
  type: FLOAT
}
attribute {
  name: "beta"
  f: 1.0
  type: FLOAT
}
attribute {
  name: "transB"
  i: 1
  type: INT
}
, input: "9"
output: "10"
name: "Relu_3"
op_type: "Relu"
, input: "10"
input: "mu.2.weight"
input: "mu.2.bias"
output: "11"
name: "Gemm_4"
op_type: "Gemm"
attribute {
  name: "alpha"
  f: 1.0
  type: FLOAT
}
attribute {
  name: "beta"
  f: 1.0
  type: FLOAT
}
attribute {
  name: "transB"
  i: 1
  type: INT
}
, input: "11"
output: "12"
name: "Relu_5"
op_type: "Relu"
, input: "12"
input: "mu.4.weight"
input: "mu.4.bias"
output: "13"
name: "Gemm_6"
op_type: "Gemm"
attribute {
  name: "alpha"
  f: 1.0
  type: FLOAT
}
attribute {
  nam

In [ ]:
np.array2string()